In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import numpy as np 
import pandas as pd
import csv
import time
import datetime
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
    
# 准备数据
data_url = '../data/more_TTI_preprosess.csv' # 构建的历史TTI数据特征
train_df = pd.read_csv(data_url, infer_datetime_format=True, parse_dates=['time'])
data_url = '../data/gps_features_detailed.csv' # 构建的GPS数据特征
gps_df = pd.read_csv(data_url, infer_datetime_format=True, parse_dates=['time'])
train_df = train_df.merge(gps_df, how='left', left_on='time', right_on='time')
train_df.fillna(0,inplace=True)
# train_df = train_df[train_df['time'].apply(lambda t: t <= datetime.datetime.strptime('2019-12-20 23:59:59', "%Y-%m-%d %H:%M:%S"))]

road_id = [276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]

def find_festival(Datetime, how):
    day_length = timedelta(days = 999)
    if how=='last':# 寻找前一个距离最近的节假日
        for days in holidays.keys():
            if days <= Datetime.date() and Datetime.date()-days<day_length:
                day_length = Datetime.date()-days
                length = day_length.days
                date = days
    elif how=='next':# 寻找后一个距离最近的节假日
        for days in holidays.keys():
            if days > Datetime.date() and days - Datetime.date()<day_length:
                day_length = days - Datetime.date()
                length = day_length.days
                date = days
    return length, holidays[date]

# 1月1日元旦、2月4日除夕、2月5日-2月10日春节、2月11日开工日、2月19元宵节、10月1-7日国庆节
# 节假日及其假期的天数
holidays = {datetime.date(2018,12,30):3, datetime.date(2019,2,4):7, datetime.date(2019,10,1):7
           ,datetime.date(2020,1,1):1, datetime.date(2020,1,17):7, datetime.date(2020,1,24):7, datetime.date(2020,4,4):3}


Y_labels = ['TTI_0','TTI_1','TTI_2','recent'] # y的标签
X_labels = ['last_festival', 'last_festival_days' ,'next_festival', 'next_festival_days']

# 加入周期性的信息和节假日的信息
train_df['month'] = train_df['time'].apply(lambda x:x.month)
train_df['day'] = train_df['time'].apply(lambda x:x.day) # 该月中的第几天
train_df['hour'] = train_df['time'].apply(lambda x:x.hour)
train_df['minute'] = train_df['time'].apply(lambda x:x.minute)
train_df['week'] = train_df['time'].apply(lambda x:x.dayofweek)
train_df['weekday'] = train_df['time'].apply(lambda x:x.weekday())
train_df['n_10'] = train_df['hour'] * 6 + train_df['minute'] // 10 + 1  #记录当天第几个10分钟
train_df['is_festival'] = train_df['time'].apply(lambda t: 0 if (t.date() not in holidays.keys()) else 1) #标示：是否是节假日
train_df['last_festival'] = train_df['time'].apply(lambda t: find_festival(t, how='last')[0]) #距离前一个节假日天数
train_df['last_festival_days'] = train_df['time'].apply(lambda t: find_festival(t, how='last')[1]) #前一个节假日天数
train_df['next_festival'] = train_df['time'].apply(lambda t: find_festival(t, how='next')[0]) #距离后一个节假日天数
train_df['next_festival_days'] = train_df['time'].apply(lambda t: find_festival(t, how='next')[1]) #前一个节假日天数

to_predict_X_test = pd.read_csv('./to_predict_x_gps_detailed3.csv', infer_datetime_format=True, parse_dates=['time'])
to_predict_X_test = to_predict_X_test[:84]
to_predict_X_test.fillna(0,inplace=True)
to_predict_X_test['month'] = to_predict_X_test['time'].apply(lambda x:x.month)
to_predict_X_test['day'] = to_predict_X_test['time'].apply(lambda x:x.day) # 该月中的第几天
to_predict_X_test['hour'] = to_predict_X_test['time'].apply(lambda x:x.hour)
to_predict_X_test['minute'] = to_predict_X_test['time'].apply(lambda x:x.minute)
to_predict_X_test['week'] = to_predict_X_test['time'].apply(lambda x:x.dayofweek)
to_predict_X_test['weekday'] = to_predict_X_test['time'].apply(lambda x:x.weekday())
to_predict_X_test['is_festival'] = to_predict_X_test['time'].apply(lambda t: 0 if (t.date() not in holidays.keys()) else 1) #标示：是否是节假日
to_predict_X_test['n_10'] = to_predict_X_test['hour'] * 6 + to_predict_X_test['minute'] // 10 + 1  #记录当天第几个10分钟
to_predict_X_test['last_festival'] = to_predict_X_test['time'].apply(lambda t: find_festival(t, how='last')[0]) #距离前一个节假日天数
to_predict_X_test['last_festival_days'] = to_predict_X_test['time'].apply(lambda t: find_festival(t, how='last')[1]) #前一个节假日天数
to_predict_X_test['next_festival'] = to_predict_X_test['time'].apply(lambda t: find_festival(t, how='next')[0]) #距离后一个节假日天数
to_predict_X_test['next_festival_days'] = to_predict_X_test['time'].apply(lambda t: find_festival(t, how='next')[1]) #前一个节假日天数


to_predict_X = pd.read_csv('./to_predict_x_gps_detailed3_stage2.csv', infer_datetime_format=True, parse_dates=['time'])
to_predict_X.fillna(0,inplace=True)
to_predict_X['month'] = to_predict_X['time'].apply(lambda x:x.month)
to_predict_X['day'] = to_predict_X['time'].apply(lambda x:x.day) # 该月中的第几天
to_predict_X['hour'] = to_predict_X['time'].apply(lambda x:x.hour)
to_predict_X['minute'] = to_predict_X['time'].apply(lambda x:x.minute)
to_predict_X['week'] = to_predict_X['time'].apply(lambda x:x.dayofweek)
to_predict_X['weekday'] = to_predict_X['time'].apply(lambda x:x.weekday())
to_predict_X['is_festival'] = to_predict_X['time'].apply(lambda t: 0 if (t.date() not in holidays.keys()) else 1) #标示：是否是节假日
to_predict_X['n_10'] = to_predict_X['hour'] * 6 + to_predict_X['minute'] // 10 + 1  #记录当天第几个10分钟
to_predict_X['last_festival'] = to_predict_X['time'].apply(lambda t: find_festival(t, how='last')[0]) #距离前一个节假日天数
to_predict_X['last_festival_days'] = to_predict_X['time'].apply(lambda t: find_festival(t, how='last')[1]) #前一个节假日天数
to_predict_X['next_festival'] = to_predict_X['time'].apply(lambda t: find_festival(t, how='next')[0]) #距离后一个节假日天数
to_predict_X['next_festival_days'] = to_predict_X['time'].apply(lambda t: find_festival(t, how='next')[1]) #前一个节假日天数

known_21 = pd.read_csv('1221known.csv')['TTI']
known_0101 = pd.read_csv('0101known.csv')['TTI']


keypoints = [[114.01619, 22.59639],[114.02036, 22.58824],[114.03132, 22.59794],
             [114.01962, 22.60689],[114.03029, 22.60736],[114.02983, 22.61313],
             [114.03332, 22.60873],[114.02789, 22.60277],[114.02570, 22.60831],
             [114.02594, 22.61078],[114.03243, 22.59199],[114.02634, 22.61345],
             [114.033524, 22.605982],[114.01562, 22.59151],[114.02367, 22.58933],
            [114.028857, 22.604546],[114.02996, 22.61016]]
keydirections = [[[0,90],[180,270]],[[45,90],[225,270]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,90],[180,270]],[[0,90],[180,270]],
                [[0,90],[180,270]],[[90,270],[270,90]],[[90,270],[270,90]],
                [[0,90],[180,270]],[[0,180],[180,360]],[[90,270],[270,90]],
                 [[90,180],[270,360]],[[0,90],[180,270]],[[45,90],[225,270]],
                 [[0,90],[180,270]],[[90,180],[270,360]]]

# 差分特征构建
for rid in road_id:
    for minutes in range(10,60,10):
        train_df['speed{}_diff{}'.format(rid,minutes)] = train_df['speed_{}_{}'.format(rid,minutes)]-train_df['speed_{}_{}'.format(rid,minutes-10)]
        train_df['tti{}_diff{}'.format(rid,minutes)] = train_df['tti_{}_{}'.format(rid,minutes)]-train_df['tti_{}_{}'.format(rid,minutes-10)]
        to_predict_X['speed{}_diff{}'.format(rid,minutes)] = to_predict_X['speed_{}_{}'.format(rid,minutes)]-to_predict_X['speed_{}_{}'.format(rid,minutes-10)]
        to_predict_X['tti{}_diff{}'.format(rid,minutes)] = to_predict_X['tti_{}_{}'.format(rid,minutes)]-to_predict_X['tti_{}_{}'.format(rid,minutes-10)]
        to_predict_X_test['speed{}_diff{}'.format(rid,minutes)] = to_predict_X_test['speed_{}_{}'.format(rid,minutes)]-to_predict_X_test['speed_{}_{}'.format(rid,minutes-10)]
        to_predict_X_test['tti{}_diff{}'.format(rid,minutes)] = to_predict_X_test['tti_{}_{}'.format(rid,minutes)]-to_predict_X_test['tti_{}_{}'.format(rid,minutes-10)]
n_keypoints = len(keypoints)
for minutes in range(10,60,10):
    for i in range(n_keypoints):
        for j in range(len(keydirections[i])):# 每个观测点不同的方向
            for minutes in range(10, 60, 5):
                train_df['point{}direction{}volume_diff{}'.format(i, j, minutes)] = train_df['point{}direction{}volume{}'.format(i, j, minutes)] - train_df['point{}direction{}volume{}'.format(i, j, minutes-5)]
                train_df['point{}direction{}speedmean_diff{}'.format(i, j, minutes)] = train_df['point{}direction{}speedmean{}'.format(i, j, minutes)] - train_df['point{}direction{}speedmean{}'.format(i, j, minutes-5)]
                to_predict_X['point{}direction{}volume_diff{}'.format(i, j, minutes)] = to_predict_X['point{}direction{}volume{}'.format(i, j, minutes)] - to_predict_X['point{}direction{}volume{}'.format(i, j, minutes-5)]
                to_predict_X['point{}direction{}speedmean_diff{}'.format(i, j, minutes)] = to_predict_X['point{}direction{}speedmean{}'.format(i, j, minutes)] - to_predict_X['point{}direction{}speedmean{}'.format(i, j, minutes-5)]
                to_predict_X_test['point{}direction{}volume_diff{}'.format(i, j, minutes)] = to_predict_X_test['point{}direction{}volume{}'.format(i, j, minutes)] - to_predict_X_test['point{}direction{}volume{}'.format(i, j, minutes-5)]
                to_predict_X_test['point{}direction{}speedmean_diff{}'.format(i, j, minutes)] = to_predict_X_test['point{}direction{}speedmean{}'.format(i, j, minutes)] - to_predict_X_test['point{}direction{}speedmean{}'.format(i, j, minutes-5)]
 


In [ ]:
parameter_table = [
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
    [{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'},{'boosting_type': 'gbdt', 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 50, 'objective': 'regression_l1'}],
]


related_table = [
    [[276183, 276184, 275911, 275912, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,1,2,4,5,7,10,12,13,14,15]],
    [[276183, 276184, 275911, 275912, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,1,2,4,5,7,10,12,13,14,15]],
    [[276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276737, 276738]
     ,[0,1,2,3,4,7,8,9,10,13,14,15]],
    [[276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276737, 276738]
     ,[0,1,2,3,4,7,8,9,10,13,14,15]],
    [[275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,2,3,4,5,6,7,8,9,11,12,13,15,16]],
    [[275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,2,3,4,5,6,7,8,9,11,12,13,15,16]],
    [[276183, 276184, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]],
    [[276183, 276184, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]],
    [[276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[1,2,3,4,5,6,7,8,9,10,11,12,14,15,16]],
    [[276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[1,2,3,4,5,6,7,8,9,10,11,12,14,15,16]],
    [[276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]],
    [[276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
     ,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]],
]


In [ ]:
road_id = [276183, 276184, 275911, 275912, 276240, 276241, 276264, 276265, 276268, 276269, 276737, 276738]
predictions = []
predictions2 = []
maes_1221 = []
maes_0101 = []
new_order = [2,3,0,1,4,5,6,7,8,9,10,11]
for road_num in range(len(road_id)):
    train_df_temp = train_df[(train_df['road_id'] == road_id[road_num]) & (train_df['n_10'] >= 45) & (train_df['n_10'] <= 121)]
    # 可以调整看使用speed还是TTI预测
    Y_labels = ['TTI_0','TTI_1','TTI_2','recent'] # y的标签
    X_labels = ['last_festival', 'last_festival_days' ,'next_festival', 'next_festival_days', 'n_10']
    for rid in related_table[road_num][0]:
        for minutes in range(0,60,10):
            X_labels.append('speed_{}_{}'.format(rid,minutes))
#             X_labels.append('tti_{}_{}'.format(rid,minutes))

    # 加入GPS采样点的特征信息
    n_keypoints = len(keypoints)
    for i in related_table[road_num][1]:
        for j in range(len(keydirections[i])):# 每个观测点不同的方向
            for minutes in range(0, 60, 5):
                X_labels.append('point{}direction{}volume{}'.format(i, j, minutes))
                X_labels.append('point{}direction{}speedmean{}'.format(i, j, minutes))
    #             train_df['point{}direction{}speedmean{}'.format(i, j, minutes)].replace(0, 40, inplace=True)
    #             to_predict_X['point{}direction{}speedmean{}'.format(i, j, minutes)].replace(0, 40, inplace=True)

    # 构建差分特征
#     for rid in road_id:
    rid = road_id[road_num]
    for minutes in range(10,60,10):
        X_labels.append('speed{}_diff{}'.format(rid,minutes))
#         X_labels.append('tti{}_diff{}'.format(rid,minutes))
        
    # for i in range(n_keypoints):
    #     for j in range(len(keydirections[i])):# 每个观测点不同的方向
    #         for minutes in range(10, 60, 5):
    #             X_labels.append('point{}direction{}volume_diff{}'.format(i, j, minutes))
    #             X_labels.append('point{}direction{}speedmean_diff{}'.format(i, j, minutes))

    X, Y = train_df_temp[X_labels],train_df_temp[Y_labels]
    # 参数可以调boosting, objective
    params = {
    'boosting_type': 'gbdt', 'max_depth': 50, 'n_estimators': 200, 'num_leaves': 30, 'objective': 'regression_l1'}
    
    # 在此处可以修改使用差分预测还是非差分，还需要将👇的y_prediction.append(y_10[i] + temp.iloc[i,2])的加法删去
    data = lgb.Dataset(X, label=Y['TTI_0'])
    lgb1s = lgb.train(parameter_table[road_num][0], data, feature_name=X_labels)

    data = lgb.Dataset(X, label=Y['TTI_1'])
    lgb2s = lgb.train(parameter_table[road_num][1], data, feature_name=X_labels)

    data = lgb.Dataset(X, label=Y['TTI_2'])
    lgb3s = lgb.train(parameter_table[road_num][2], data, feature_name=X_labels)
#     print('road {} train MAE ='.format(road_id[road_num]), (mean_absolute_error(Y['TTI_0'], lgb1s.predict(X)) + mean_absolute_error(Y['TTI_1'], lgb2s.predict(X)) + mean_absolute_error(Y['TTI_2'], lgb3s.predict(X)))/3)


    temp = to_predict_X[to_predict_X['road_id']==road_id[road_num]]
    temp2 = to_predict_X_test[to_predict_X_test['road_id']==road_id[road_num]]
    y_prediction = []
    y_prediction2 = []
    y_10 = lgb1s.predict(temp[X_labels])
    y_20 = lgb2s.predict(temp[X_labels])
    y_30 = lgb3s.predict(temp[X_labels])
    y2_10 = lgb1s.predict(temp2[X_labels])
    y2_20 = lgb2s.predict(temp2[X_labels])
    y2_30 = lgb3s.predict(temp2[X_labels])
    for i in range(len(y_10)):
        y_prediction.append(y_10[i])
        y_prediction.append(y_20[i])
        y_prediction.append(y_30[i])
    for i in range(len(y2_10)):
        y_prediction2.append(y2_10[i])
        y_prediction2.append(y2_20[i])
        y_prediction2.append(y2_30[i])
    
    test_y = known_0101[list(range(21*road_num, 21*road_num+21))]
    test_y2 = known_21[list(range(21*road_num, 21*road_num+21))]
    mae = mean_absolute_error(test_y, y_prediction[:len(test_y)])
    mae2 = mean_absolute_error(test_y2, y_prediction2[:len(test_y2)])
    predictions.append(y_prediction)
    predictions2.append(y_prediction2)
    maes_0101.append(mae)
    maes_1221.append(mae2)
#     print('MAE in 01.01 =', mae)
#     print('MAE in 12.21 =', mae2)
print(np.mean(maes_0101))    
print(np.mean(maes_1221))


# 输出结果到csv
y_prediction = []
for day in range(12):
    for road_num in [2,3,0,1,4,5,6,7,8,9,10,11]:
        y_prediction = y_prediction + predictions[road_num][day*21:(day+1)*21]
csvFile = open("lgb_l1/lgb_speed_nodiff_l1.csv", "w")
writer = csv.writer(csvFile)
writer.writerow(["id_sample", "TTI"])
for i in range(len(y_prediction)):
    writer.writerow([i, y_prediction[i]])
csvFile.close()
print("Finished prediction!")


